# UChicago - ML2 Final Project
# Author: Akanksha Mathpati, Chris Tomaszkiewicz, Deepali Dagar, Sakshi Bokil
# <b> Streamlit Interface
(Uses recommendation model)

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
cd /content/drive/MyDrive/UOC/ML2/Resume dataset/

/content/drive/MyDrive/UOC/ML2/Resume dataset


## Import Libraries

In [2]:
!pip install streamlit pyngrok --quiet
import spacy
from sentence_transformers import SentenceTransformer
import streamlit as st
import joblib
import pandas as pd
import numpy as np
import spacy
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from io import StringIO
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.2 MB/s eta 0:00:00


<b> Using NGROK for hosting website

In [4]:
!ngrok config add-authtoken 2wfW2U1STuVaKcQS0rDvFDcwk11_6wvocPc7L16fdynLEoBa42

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [149]:
ls

 app.py              job_embeddings.npy            nlp_custom/
 cleaned_data.csv    jobs/                         postings.csv
 combined_data.csv  'jobseeker (1).jpg'            Resume.csv
 companies/          jobseeker.jpg                 resume_embeddings.npy
'employer (1).jpg'   mappings/                     Resume-job/
 employer.jpg        matcher.pkl                   sbert_model/
 illustration.jpeg   mlb.pkl                       vectorizer.pkl
 integrated_app.py   ML_Project_classifier.ipynb   xgb_model.pkl
'job1 (1).jpg'       models_bundle/
 job1.jpg            models_bundle.zip


In [187]:
from google.colab import files
uploaded = files.upload()


Saving employer.jpg to employer.jpg
Saving jobseeker.jpg to jobseeker.jpg


In [67]:
# To kill instances 
!pkill streamlit
!streamlit run app.py &>/dev/null &
!pkill ngrok

In [70]:
%%writefile app.py

try:
    from PyPDF2 import PdfReader
except ImportError:
    PdfReader = None

try:
    import docx
except ImportError:
    docx = None

_drive_path = "/content/drive/MyDrive/UOC/ML2/Resume dataset/"


# Define functions to use
class Matcher:
    def __init__(self, model_path, nlp_path=None):
        self.model = SentenceTransformer(model_path)
        try:
            self.nlp = spacy.load("en_core_web_sm") if nlp_path is None else spacy.load(nlp_path)
        except:
            import spacy.cli
            spacy.cli.download("en_core_web_sm")
            self.nlp = spacy.load("en_core_web_sm")

    def preprocess(self, text):
        doc = self.nlp(text.lower())
        return " ".join([t.lemma_ for t in doc if not t.is_stop and not t.is_punct])

    def encode(self, texts):
        return self.model.encode(texts, show_progress_bar=False)

# Page Configuration 
st.set_page_config(page_title="HireSense", layout="wide")

# Load the models 
@st.cache_resource
def load_assets():
    try:
        model = joblib.load(os.path.join(_drive_path, "xgb_model.pkl"))
        vectorizer = joblib.load(os.path.join(_drive_path, "vectorizer.pkl"))
        mlb = joblib.load(os.path.join(_drive_path, "mlb.pkl"))
        matcher = Matcher(os.path.join(_drive_path, "sbert_model"))
        return model, vectorizer, mlb, matcher
    except Exception as e:
        st.error(f"Error loading models: {str(e)}")
        return None, None, None, None

model, vectorizer, mlb, matcher = load_assets()

# Session State Initialization
if "results" not in st.session_state:
    st.session_state.results = {}
if "role" not in st.session_state:
    st.session_state.role = None

# UI Styles
st.markdown("""
    <style>
    .stApp { background-color: #f8f9fa; }
    .block-container { background-color: rgba(255,255,255,0.95); }
    .role-card {
        border: 2px solid #4FC3F7;
        border-radius: 10px;
        padding: 2rem;
        margin: 1rem;
        transition: all 0.3s ease;
    }
    .role-card:hover {
        transform: translateY(-5px);
        box-shadow: 0 8px 16px rgba(0,0,0,0.1);
    }
    .prediction-badge {
        background: #0288D1;
        color: white;
        padding: 8px 16px;
        border-radius: 20px;
        margin: 4px;
    }
    </style>
""", unsafe_allow_html=True)

# Role Selection
if not st.session_state.role:
    st.title("HireSense")
    st.markdown("### Select Your Profile")
    col1, col2 = st.columns(2)
    with col1:
        with st.form("seeker_form"):
            st.markdown("""
                <div class='role-card'>
                    <h3 style='text-align: center'>Job Seeker</h3>
                    <p style='text-align: center'>Upload resume to find matching jobs</p>
                </div>
            """, unsafe_allow_html=True)
            if st.form_submit_button("Continue as Job Seeker"):
                st.session_state.role = "seeker"
    with col2:
        with st.form("employer_form"):
            st.markdown("""
                <div class='role-card'>
                    <h3 style='text-align: center'>Employer</h3>
                    <p style='text-align: center'>Post job to find candidates</p>
                </div>
            """, unsafe_allow_html=True)
            if st.form_submit_button("Continue as Employer"):
                st.session_state.role = "employer"

# Main Interface
else:
    if st.button("← Change Role"):
        st.session_state.clear()
        st.experimental_rerun()

    st.title(f"{'Job Seeker' if st.session_state.role == 'seeker' else 'Employer'} Portal")

    if st.session_state.role == "seeker":
        uploaded_file = st.file_uploader("Upload Resume (PDF/DOCX/TXT)", type=["pdf", "docx", "txt"])
        resume_text = ""
        if uploaded_file:
            try:
                if uploaded_file.type == "application/pdf":
                    reader = PdfReader(uploaded_file)
                    resume_text = "\n".join([p.extract_text() or "" for p in reader.pages])
                elif uploaded_file.type == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
                    doc = docx.Document(uploaded_file)
                    resume_text = "\n".join([p.text for p in doc.paragraphs])
                elif uploaded_file.type == "text/plain":
                    resume_text = uploaded_file.getvalue().decode()
            except Exception as e:
                st.error(f"Error reading file: {str(e)}")
        resume_input = st.text_area("Or paste resume text:", value=resume_text, height=200)
        if st.button("Find Matching Jobs"):
            if not resume_input.strip():
                st.warning("Please input resume text")
            else:
                with st.spinner("Finding best matches..."):
                    try:
                        cleaned = matcher.preprocess(resume_input)
                        resume_emb = matcher.encode([cleaned])
                        job_df = pd.read_csv(os.path.join(_drive_path, "postings.csv"))
                        job_embs = np.load(os.path.join(_drive_path, "job_embeddings.npy"))
                        if resume_emb.ndim == 1:
                            resume_emb = resume_emb.reshape(1, -1)
                        if job_embs.ndim == 1:
                            job_embs = job_embs.reshape(1, -1)
                        scores = cosine_similarity(resume_emb, job_embs)[0]
                        top_indices = scores.argsort()[::-1][:5]
                        st.session_state.results = {
                            "matches": {
                                "data": job_df.iloc[top_indices],
                                "scores": scores[top_indices]
                            }
                        }
                    except Exception as e:
                        st.error(f"Processing error: {str(e)}")

    elif st.session_state.role == "employer":
        st.header("Post a Job Description")
        jd_input = st.text_area("Paste the job description here:")
        if st.button("Find Matching Resumes"):
            if not jd_input.strip():
                st.warning("Please paste a job description to find candidates.")
            else:
                try:
                    cleaned = matcher.preprocess(jd_input)
                    jd_emb = matcher.encode([cleaned])
                    resume_df = pd.read_csv(os.path.join(_drive_path, "Resume.csv"))
                    resume_embs = np.load(os.path.join(_drive_path, "resume_embeddings.npy"))
                    if jd_emb.ndim == 1:
                        jd_emb = jd_emb.reshape(1, -1)
                    if resume_embs.ndim == 1:
                        resume_embs = resume_embs.reshape(1, -1)
                    scores = cosine_similarity(jd_emb, resume_embs)[0]
                    top_indices = scores.argsort()[::-1][:5]
                    st.subheader("Top Resume Matches")
                    for idx in top_indices:
                        resume = resume_df.iloc[idx]
                        st.markdown(f"**Resume ID:** `{resume['ID']}`")
                        st.markdown(f"**Snippet:** {resume['Resume_str'][:300]}...")
                        st.markdown(f"**Similarity Score:** `{scores[idx]:.4f}`")
                        st.write("---")
                except Exception as e:
                    st.error(f"Error during resume matching: {e}")

    # Results Display for Job Seeker
    if st.session_state.role == "seeker" and "matches" in st.session_state.results:
        st.subheader("Top Job Matches")
        matches = st.session_state.results["matches"]
        for idx, (score, row) in enumerate(zip(matches["scores"], matches["data"].itertuples())):
            with st.expander(f"Match {idx+1} | Score: {score:.2f}", expanded=True):
                st.markdown(f"**{row.title}**")
                st.write(f"**ID:** {row.job_id}")
                st.write("---")


Overwriting app.py


In [71]:
# To run the website
if 'google.colab' in str(os.sys.modules):
    from pyngrok import ngrok
    import threading

    def run_streamlit():
        os.system("streamlit run app.py --server.port 8501")

    threading.Thread(target=run_streamlit).start()
    public_url = ngrok.connect(8501)
    print(f"App running at: {public_url}")

🚀 App running at: NgrokTunnel: "https://9c83-34-169-230-218.ngrok-free.app" -> "http://localhost:8501"
